<a href="https://colab.research.google.com/github/AKASH4671/Sentiment-Analysis-on-Financial-News-and-Its-Impact-on-Stock-Prices/blob/main/01_fetch_news.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**mount drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**libraries**

In [ ]:
!pip install requests
import requests
import pandas as pd
from datetime import datetime, timedelta
from time import sleep

**1 day**

In [ ]:
# Add NewsAPI key here
API_KEY = "add API key"

# Parameters
TICKER = "AAPL"
QUERY = "Apple"
FROM_DATE = (datetime.now() - timedelta(days=30)).strftime('%Y-%m-%d')
TO_DATE = datetime.now().strftime('%Y-%m-%d')
PAGE_SIZE = 100  # max per page
MAX_PAGES = 5    # fetch up to 500 articles

# Endpoint
URL = "https://newsapi.org/v2/everything"

articles = []

print(f" Fetching news for {QUERY} from {FROM_DATE} to {TO_DATE}...")

for page in range(1, MAX_PAGES + 1):
    params = {
        "q": QUERY,
        "from": FROM_DATE,
        "to": TO_DATE,
        "sortBy": "publishedAt",
        "language": "en",
        "pageSize": PAGE_SIZE,
        "page": page,
        "apiKey": API_KEY
    }

    response = requests.get(URL, params=params)
    data = response.json()

    if response.status_code != 200 or "articles" not in data:
        print(f" Error fetching page {page}:", data.get("message"))
        break

    print(f" Page {page}: {len(data['articles'])} articles")

    for item in data["articles"]:
        articles.append({
            "date": item["publishedAt"],
            "headline": item["title"],
            "summary": item["description"],
            "text": item["content"],
            "url": item["url"],
            "source": item["source"]["name"]
        })

    if len(data["articles"]) < PAGE_SIZE:
        break  # no more data

 Fetching news for Apple from 2025-06-23 to 2025-07-23...
 Page 1: 98 articles


In [ ]:
# Save
df = pd.DataFrame(articles)
df.to_csv("/content/drive/MyDrive/Colab Notebooks/Finance Projects/Sentiment-Analysis-on-Financial-News-and-Its-Impact-on-Stock-Prices/data/stock_news_for_1_day.csv", index=False)
print(f"\n Saved {len(df)} articles to 'data/stock_news.csv'")


 Saved 98 articles to 'data/stock_news.csv'


In [ ]:
df.head()

,date,headline,summary,text,url,source
0,2025-07-22T17:03:13Z,"Download: iOS 26 Beta 4 OTA Profile Update, IP...",Looking to download the latest iOS 26 beta 4 v...,Looking to download the latest iOS 26 beta 4 v...,https://www.redmondpie.com/download-ios-26-bet...,redmondpie.com
1,2025-07-22T17:02:40Z,Apple Seeds Fourth Developer Betas of iOS 26 a...,Apple today provided developers with the fourt...,Apple today provided developers with the fourt...,https://www.macrumors.com/2025/07/22/apple-see...,MacRumors
2,2025-07-22T17:02:33Z,Apple TV+ comedy ‘Loot’ to return for season t...,Apple TV+ on Tuesday revealed that its comedy ...,"Maya Rudolph in Loot season three, premiering ...",https://macdailynews.com/2025/07/22/apple-tv-c...,Macdailynews.com
3,2025-07-22T17:02:27Z,Apple releases iOS 26 beta 4 for iPhone,Apple changed a few Liquid Glass elements in i...,Apple changed a few Liquid Glass elements in i...,https://9to5mac.com/2025/07/22/ios-26-beta-4/,9to5Mac
4,2025-07-22T17:02:22Z,iPadOS 26 beta 4 now available with latest iPa...,Apple has just released iPadOS 26 beta 4 to de...,Apple has just released iPadOS 26 beta 4 to de...,https://9to5mac.com/2025/07/22/ipados-26-beta-...,9to5Mac


In [ ]:
# Convert 'date' to datetime
df['date'] = pd.to_datetime(df['date'])

# Show earliest and latest date
print(" News Date Range:")
print("Start:", df['date'].min())
print("End:  ", df['date'].max())

 News Date Range:
Start: 2025-07-22 15:28:47+00:00
End:   2025-07-22 17:03:13+00:00


**30days**

In [ ]:
# API Key
API_KEY = "add API key"

#  Parameters
company = "Apple"
from_date = datetime(2025, 6, 23)
to_date = datetime(2025, 7, 23)
all_articles = []

print(f" Fetching news for {company} from {from_date.date()} to {to_date.date()}...")

#  Loop through each day
current_date = from_date
while current_date <= to_date:
    next_day = current_date + timedelta(days=1)
    page = 1

    print(f"\n Date: {current_date.date()}")

    while True:
        url = (
            f"https://newsapi.org/v2/everything?q={company}&from={current_date.date()}&to={next_day.date()}"
            f"&sortBy=publishedAt&pageSize=100&page={page}&apiKey={API_KEY}"
        )
        response = requests.get(url)
        data = response.json()

        if response.status_code != 200 or data.get("status") != "ok":
            print(f" Error on {current_date.date()} page {page}: {data.get('message', 'Unknown error')}")
            break

        articles = data.get("articles", [])
        if not articles:
            break

        print(f"Page {page}: {len(articles)} articles")
        for article in articles:
            all_articles.append({
                "date": article["publishedAt"],
                "headline": article["title"],
                "summary": article["description"],
                "url": article["url"],
                "source": article["source"]["name"],
                "content": article.get("content", "")
            })

        if len(articles) < 100:
            break  # Last page
        page += 1
        sleep(1)  # Respect rate limits

    current_date += timedelta(days=1)
    sleep(0.5)  # Light pause between days



 Fetching news for Apple from 2025-06-23 to 2025-07-23...

 Date: 2025-06-23
Page 1: 98 articles

 Date: 2025-06-24
Page 1: 98 articles

 Date: 2025-06-25
Page 1: 100 articles
 Error on 2025-06-25 page 2: You have requested too many results. Developer accounts are limited to a max of 100 results. You are trying to request results 100 to 200. Please upgrade to a paid plan if you need more results.

 Date: 2025-06-26
Page 1: 100 articles
 Error on 2025-06-26 page 2: You have requested too many results. Developer accounts are limited to a max of 100 results. You are trying to request results 100 to 200. Please upgrade to a paid plan if you need more results.

 Date: 2025-06-27
Page 1: 98 articles

 Date: 2025-06-28
Page 1: 98 articles

 Date: 2025-06-29
Page 1: 100 articles
 Error on 2025-06-29 page 2: You have requested too many results. Developer accounts are limited to a max of 100 results. You are trying to request results 100 to 200. Please upgrade to a paid plan if you need more res

In [ ]:
#  Create DataFrame
df_news = pd.DataFrame(all_articles)
df_news['date'] = pd.to_datetime(df_news['date'])


output_path = "/content/drive/MyDrive/Colab Notebooks/Finance Projects/Sentiment-Analysis-on-Financial-News-and-Its-Impact-on-Stock-Prices/data/apple_newsapi_news.csv"
df_news.to_csv(output_path, index=False)
print(f"\n Saved {len(df_news)} articles to: {output_path}")

#  Check date range
print("\n Date range in collected data:")
print("Start:", df_news['date'].min())
print("End:  ", df_news['date'].max())



 Saved 2948 articles to: /content/drive/MyDrive/Colab Notebooks/Finance Projects/Sentiment-Analysis-on-Financial-News-and-Its-Impact-on-Stock-Prices/data/apple_newsapi_news.csv

 Date range in collected data:
Start: 2025-06-24 21:05:19+00:00
End:   2025-07-22 23:58:03+00:00


In [ ]:
df_news.head()

,date,headline,summary,url,source,content
0,2025-06-24 23:59:58+00:00,Node v22.17.0 (LTS),"Node.js® is a free, open-source, cross-platfor...",https://nodejs.org/en/blog/release/v22.17.0,Nodejs.org,Antoine du Hamel\r\nConstructing classes like ...
1,2025-06-24 23:57:16+00:00,"Marcelo vê Moçambique a ""olhar para o futuro""",Presidente da República está em Moçambique par...,https://observador.pt/2025/06/25/marcelo-ve-mo...,Observador.pt,O Presidente da República Portuguesa disse est...
2,2025-06-24 23:56:34+00:00,Obediências maçónicas emitem declaração em def...,"Cabecinha, Rola e Valente alertam para esvazia...",https://observador.pt/2025/06/25/tres-obedienc...,Observador.pt,Os dirigentes das três principais obediências ...
3,2025-06-24 23:54:57+00:00,8点1氪｜顺丰等多家快递公司拒收罗马仕充电宝；字节通报大模型团队负责人出轨HRBP处理结果；...,上市进行时\n 斯坦德机器人（无锡）股份有限公司\n 36氪获悉，近日，斯坦德机器人（无锡）...,https://36kr.com/p/3351137481288323,36kr.com,36App201662210.52\r\n36623\r\n36HH\r\nTOP3\r\n...
4,2025-06-24 23:54:56+00:00,Almada. Detido suspeito de tentativa de homicí...,"Cidadão luso-brasileiro de 48 anos efetuou, em...",https://observador.pt/2025/06/25/detido-em-alm...,Observador.pt,"A Polícia Judiciária (PJ) deteve em Almada, no..."


In [ ]:
df_news.shape

(2948, 6)